# Debug Drill: The Expensive Threshold

**Scenario:**
A colleague built a churn prediction model and deployed it with the default threshold of 0.5.

"It works great!" they say. "We're catching churners!"

But the CFO is unhappy: "Our retention campaign costs are through the roof!"

**Your Task:**
1. Run the current model and understand the cost structure
2. Find why the threshold is suboptimal
3. Fix it to minimize business cost
4. Write a 3-bullet postmortem

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
# Load data and train model
DATA_URL = 'https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/'

customers = pd.read_csv(DATA_URL + 'streamcart_customers.csv')
print(f"Loaded {len(customers)} customers")
print(f"Churn rate: {customers['churn_30d'].mean():.1%}")

In [ ]:
# Prepare features
if 'tenure_days' not in customers.columns:
    customers['tenure_days'] = (pd.to_datetime('2024-01-01') - pd.to_datetime(customers['signup_date'])).dt.days
if 'avg_order_value' not in customers.columns:
    customers['avg_order_value'] = customers['total_spend'] / customers['orders_total'].replace(0, 1)

feature_cols = ['tenure_days', 'orders_total', 'total_spend', 'support_tickets_total', 'avg_order_value']
available_features = [c for c in feature_cols if c in customers.columns]

X = customers[available_features].fillna(0)
y = customers['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Train logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Get probabilities
probabilities = model.predict_proba(X_test)[:, 1]
print(f"Model trained. Probability range: [{probabilities.min():.2f}, {probabilities.max():.2f}]")

In [ ]:
# ===== COLLEAGUE'S CODE (CONTAINS BUG) =====

# Business costs
FP_COST = 50   # Cost of wasted retention offer (sent to non-churner)
FN_COST = 200  # Cost of lost customer (missed churner)

# Colleague uses DEFAULT threshold of 0.5
THRESHOLD = 0.5  # <-- BUG: Not optimized for cost structure!

predictions = (probabilities >= THRESHOLD).astype(int)

# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)
tn, fp, fn, tp = cm.ravel()

# Calculate total cost
total_cost = fp * FP_COST + fn * FN_COST

print("=== Colleague's Model (threshold = 0.5) ===")
print(f"\nConfusion Matrix:")
print(f"  True Negatives:  {tn:4d} (correctly identified non-churners)")
print(f"  False Positives: {fp:4d} (wasted offers @ ${FP_COST} = ${fp * FP_COST:,})")
print(f"  False Negatives: {fn:4d} (missed churners @ ${FN_COST} = ${fn * FN_COST:,})")
print(f"  True Positives:  {tp:4d} (caught churners)")
print(f"\n💰 TOTAL COST: ${total_cost:,}")
print(f"\nMetrics:")
print(f"  Precision: {precision_score(y_test, predictions):.1%}")
print(f"  Recall:    {recall_score(y_test, predictions):.1%}")

---

## Your Investigation

The CFO asks: "Can we reduce costs?"

### Step 1: Understand the cost structure

In [ ]:
# Think about the costs:
print("Cost Structure Analysis:")
print(f"  FP cost: ${FP_COST} (sending offer to non-churner)")
print(f"  FN cost: ${FN_COST} (missing a churner)")
print(f"\n  FN is {FN_COST / FP_COST}x more expensive than FP!")

print("\n🤔 Question: If missing a churner costs 4x more than a wasted offer...")
print("   Should we be MORE aggressive (lower threshold) or LESS aggressive (higher)?")

In [ ]:
# The theoretical optimal threshold formula
# For cost-sensitive classification:
#   optimal_threshold ≈ FP_cost / (FP_cost + FN_cost)

theoretical_optimal = FP_COST / (FP_COST + FN_COST)
print(f"Theoretical optimal threshold: {theoretical_optimal:.2f}")
print(f"Colleague's threshold: 0.50")
print(f"\n❌ The default 0.5 threshold is WAY too high!")
print(f"   We should use ~{theoretical_optimal:.2f} to catch more churners.")

### Step 2: Find the actual optimal threshold

In [ ]:
# TODO: Test different thresholds and find the cost-minimizing one

def calculate_cost(y_true, y_pred, fp_cost, fn_cost):
    """Calculate total business cost from predictions."""
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return fp * fp_cost + fn * fn_cost

# Uncomment and complete:

# thresholds = np.arange(0.05, 0.95, 0.05)
# results = []
# 
# for thresh in thresholds:
#     preds = (probabilities >= thresh).astype(int)
#     
#     cm = confusion_matrix(y_test, preds)
#     tn, fp, fn, tp = cm.ravel()
#     
#     cost = calculate_cost(y_test, preds, FP_COST, FN_COST)
#     prec = precision_score(y_test, preds, zero_division=0)
#     rec = recall_score(y_test, preds)
#     
#     results.append({
#         'threshold': thresh,
#         'fp': fp,
#         'fn': fn,
#         'cost': cost,
#         'precision': prec,
#         'recall': rec
#     })
# 
# results_df = pd.DataFrame(results)
# print(results_df.to_string(index=False))

In [ ]:
# TODO: Find and report the optimal threshold

# Uncomment after completing above:

# optimal_idx = results_df['cost'].idxmin()
# optimal_row = results_df.loc[optimal_idx]
# 
# print(f"\n=== OPTIMAL THRESHOLD: {optimal_row['threshold']:.2f} ===")
# print(f"  Cost: ${optimal_row['cost']:,.0f}")
# print(f"  False Positives: {optimal_row['fp']:.0f} (@ ${FP_COST} = ${optimal_row['fp'] * FP_COST:,.0f})")
# print(f"  False Negatives: {optimal_row['fn']:.0f} (@ ${FN_COST} = ${optimal_row['fn'] * FN_COST:,.0f})")
# print(f"  Precision: {optimal_row['precision']:.1%}")
# print(f"  Recall: {optimal_row['recall']:.1%}")

In [ ]:
# TODO: Calculate savings

# Uncomment:

# original_cost = total_cost  # From colleague's code
# optimal_cost = optimal_row['cost']
# savings = original_cost - optimal_cost
# 
# print(f"\n=== SAVINGS ===")
# print(f"  Original (threshold=0.5): ${original_cost:,.0f}")
# print(f"  Optimal (threshold={optimal_row['threshold']:.2f}): ${optimal_cost:,.0f}")
# print(f"  SAVINGS: ${savings:,.0f} ({savings/original_cost*100:.0f}% reduction!)")

In [ ]:
# ============================================
# SELF-CHECK: Did you find a better threshold?
# ============================================

# Uncomment after completing:

# assert optimal_row['threshold'] < 0.5, "Optimal threshold should be BELOW 0.5 (FN costs more than FP)"
# assert optimal_cost < original_cost, "Optimal cost should be lower than original"
# assert optimal_row['recall'] > recall_score(y_test, predictions), "Lower threshold should increase recall"
# 
# print("✓ Threshold optimization correct!")
# print(f"✓ Moved from 0.5 to {optimal_row['threshold']:.2f}")
# print(f"✓ Saved ${savings:,.0f}")

### Step 3: Visualize the cost curve

In [ ]:
# TODO: Visualize threshold vs cost

# Uncomment:

# fig, axes = plt.subplots(1, 2, figsize=(14, 5))
# 
# # Cost curve
# ax1 = axes[0]
# ax1.plot(results_df['threshold'], results_df['cost'], 'purple', linewidth=2)
# ax1.axvline(x=0.5, color='red', linestyle='--', label='Original (0.5)', alpha=0.7)
# ax1.axvline(x=optimal_row['threshold'], color='green', linestyle='--', 
#             label=f'Optimal ({optimal_row["threshold"]:.2f})', alpha=0.7)
# ax1.scatter([0.5], [original_cost], color='red', s=100, zorder=5)
# ax1.scatter([optimal_row['threshold']], [optimal_cost], color='green', s=100, zorder=5)
# ax1.set_xlabel('Threshold')
# ax1.set_ylabel('Total Cost ($)')
# ax1.set_title('Business Cost by Threshold')
# ax1.legend()
# ax1.grid(True, alpha=0.3)
# 
# # Precision/Recall tradeoff
# ax2 = axes[1]
# ax2.plot(results_df['threshold'], results_df['precision'], 'b-', label='Precision', linewidth=2)
# ax2.plot(results_df['threshold'], results_df['recall'], 'r-', label='Recall', linewidth=2)
# ax2.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
# ax2.axvline(x=optimal_row['threshold'], color='green', linestyle='--', alpha=0.7)
# ax2.set_xlabel('Threshold')
# ax2.set_ylabel('Score')
# ax2.set_title('Precision-Recall Tradeoff')
# ax2.legend()
# ax2.grid(True, alpha=0.3)
# 
# plt.tight_layout()
# plt.show()

### Step 4: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Expensive Threshold

### What happened:
- (Your answer: What was the symptom? What did the CFO observe?)

### Root cause:
- (Your answer: What was wrong with the threshold? Why?)

### How to prevent:
- (Your answer: What should we do before deploying classification models?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **The default threshold (0.5) is rarely optimal.** It assumes equal costs for FP and FN, which is almost never true in business.

2. **When FN >> FP cost, lower the threshold.** You want to catch more positives even at the cost of more false alarms.

3. **When FP >> FN cost, raise the threshold.** You want to be more certain before flagging positives.

4. **The formula: optimal_threshold ≈ FP_cost / (FP_cost + FN_cost)**

---

## Cost-Optimal Threshold Cheatsheet

| FP Cost | FN Cost | Optimal Threshold | Example |
|---------|---------|-------------------|----------|
| $50 | $200 | ~0.20 | Churn (losing customer is expensive) |
| $100 | $100 | ~0.50 | Balanced costs |
| $500 | $50 | ~0.91 | Spam filter (false positive is annoying) |
| $10 | $10000 | ~0.001 | Fraud detection (missing fraud is catastrophic) |